In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 16:10:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
df_green = spark.read.parquet('/home/huely/data/pq/green/*/*')

In [7]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [25]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-08 09:48:52|  2020-01-08 09:52:42|                 N|         1|          97|          49|              1|         0.82|        5.0|  0.0|    0.

In [12]:
df_green.createOrReplaceTempView('green')

In [15]:
df_green_revenue = spark.sql("""

SELECT
date_trunc( "hour", lpep_pickup_datetime) AS hour,
PULocationID AS revenue_zone,
 
COUNT(1) AS number_records,
SUM(total_amount) AS amount
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
ORDER BY
    1,2

""")

In [16]:
df_green_revenue.show()

+-------------------+------------+--------------+------------------+
|               hour|revenue_zone|number_records|            amount|
+-------------------+------------+--------------+------------------+
|2020-01-01 00:00:00|           7|            45| 769.7299999999996|
|2020-01-01 00:00:00|          17|             9|195.03000000000003|
|2020-01-01 00:00:00|          18|             1|               7.8|
|2020-01-01 00:00:00|          22|             1|              15.8|
|2020-01-01 00:00:00|          24|             3|              87.6|
|2020-01-01 00:00:00|          25|            26| 531.0000000000002|
|2020-01-01 00:00:00|          29|             1|              61.3|
|2020-01-01 00:00:00|          32|             2| 68.94999999999999|
|2020-01-01 00:00:00|          33|            11|317.27000000000004|
|2020-01-01 00:00:00|          35|             5|            129.96|
|2020-01-01 00:00:00|          36|            11| 295.3400000000001|
|2020-01-01 00:00:00|          37|

In [22]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('/home/huely/data/report/revenue/green', mode='overwrite')

In [3]:
df_yellow = spark.read.parquet('/home/huely/data/pq/yellow/*/*')

In [4]:
df_yellow.createOrReplaceTempView('yellow')

In [18]:
df_yellow_revenue = spark.sql("""

SELECT
date_trunc( "hour", tpep_pickup_datetime) AS hour,
PULocationID AS revenue_zone,
 
COUNT(1) AS number_records,
SUM(total_amount) AS amount
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2


""")

In [19]:
df_yellow_revenue.show()

+-------------------+------------+--------------+------------------+
|               hour|revenue_zone|number_records|            amount|
+-------------------+------------+--------------+------------------+
|2020-01-01 02:00:00|         234|           370| 6759.990000000011|
|2020-01-07 12:00:00|          68|           181| 3046.229999999999|
|2020-01-04 12:00:00|         249|           152| 2245.569999999999|
|2020-01-01 01:00:00|         107|           583| 9994.480000000025|
|2020-01-02 15:00:00|         236|           504|  7382.91000000002|
|2020-01-04 02:00:00|          48|           185| 3436.580000000001|
|2020-01-03 12:00:00|         163|           294| 5357.380000000009|
|2020-01-04 02:00:00|          79|           426| 6893.030000000011|
|2020-01-01 19:00:00|         100|           175|2726.7399999999966|
|2020-01-03 16:00:00|         132|           480|27982.999999999935|
|2020-01-03 18:00:00|         142|           378|  5731.95000000001|
|2020-01-06 11:00:00|         114|

In [27]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('/home/huely/data/report/revenue/yellow', mode='overwrite')

# JOIN

In [36]:
# materializing the results for effiency processing: load the previous loaded data
df_green_revenue = spark.read.parquet('/home/huely/data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('/home/huely/data/report/revenue/yellow')


In [37]:
# rename columns to seperate column names after join

df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('number_records','green_number_records') \
    .withColumnRenamed('amount','green_amount')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('number_records','yellow_number_records') \
    .withColumnRenamed('amount','yellow_amount')

In [38]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','revenue_zone'], how='outer')

In [39]:
df_join.show()

+-------------------+------------+--------------------+------------------+---------------------+------------------+
|               hour|revenue_zone|green_number_records|      green_amount|yellow_number_records|     yellow_amount|
+-------------------+------------+--------------------+------------------+---------------------+------------------+
|2020-01-01 00:00:00|           3|                NULL|              NULL|                    1|              25.0|
|2020-01-01 00:00:00|           4|                NULL|              NULL|                   57|1004.3000000000002|
|2020-01-01 00:00:00|           7|                  45| 769.7299999999996|                   38| 455.1700000000001|
|2020-01-01 00:00:00|          12|                NULL|              NULL|                    6|106.99999999999999|
|2020-01-01 00:00:00|          37|                   6|            175.67|                    7|161.60999999999999|
|2020-01-01 00:00:00|          40|                   8|168.9800000000000

In [41]:
df_join.write.parquet('/home/huely/data/report/revenue/total', mode='overwrite')

# Join one large table, one small

In [43]:
df_join = spark.read.parquet('/home/huely/data/report/revenue/total')

In [44]:
df_join.show()

+-------------------+------------+--------------------+------------------+---------------------+------------------+
|               hour|revenue_zone|green_number_records|      green_amount|yellow_number_records|     yellow_amount|
+-------------------+------------+--------------------+------------------+---------------------+------------------+
|2020-01-01 00:00:00|          14|                NULL|              NULL|                    1|               8.8|
|2020-01-01 00:00:00|          15|                NULL|              NULL|                    1|             34.09|
|2020-01-01 00:00:00|          17|                   9|195.03000000000003|                    8|220.20999999999998|
|2020-01-01 00:00:00|          25|                  26| 531.0000000000002|                   16| 324.3500000000001|
|2020-01-01 00:00:00|          32|                   2| 68.94999999999999|                    1|              18.0|
|2020-01-01 00:00:00|          43|                   6|            107.5

In [45]:
df_zones = spark.read.parquet('zones')

In [46]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [47]:
df_join

DataFrame[hour: timestamp, revenue_zone: int, green_number_records: bigint, green_amount: double, yellow_number_records: bigint, yellow_amount: double]

In [50]:
df_result = df_join.join(df_zones, df_join.revenue_zone == df_zones.LocationID)

In [53]:
df_result.printSchema()

root
 |-- hour: timestamp (nullable = true)
 |-- revenue_zone: integer (nullable = true)
 |-- green_number_records: long (nullable = true)
 |-- green_amount: double (nullable = true)
 |-- yellow_number_records: long (nullable = true)
 |-- yellow_amount: double (nullable = true)
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [57]:
df_result.drop('LocationID').write.parquet('tmp/revenue-zones')

In [58]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7fec51fc7b90>>